In [74]:
import pandas as pd

In [75]:
# File paths
snp_file = "/home/piyali/Piyali/Data/SNP/step_5_recoded_MERGED_1_GO.csv"
mri_file = "/home/piyali/Piyali/Data/Imaging/Info/MRI_T1.csv"
output_file = "/home/piyali/Piyali/Data/SNP/Subject_feature_label_SNP_final.csv"

# Step 1: Load SNP data
snp_df = pd.read_csv(snp_file, sep=" ")
snp_df = snp_df.drop(columns=['FID', 'PAT', 'MAT', 'SEX', 'PHENOTYPE'])

# Step 2: Load MRI info
mri_df = pd.read_csv(mri_file, sep=",")

In [76]:
# In mri_df all subjcts are in the form '002+AF8-S+AF8-0295', so we need to replace "+AF8-" into "_"
mri_df['Subject ID'] = mri_df['Subject ID'].astype(str).str.replace(r"\+AF8-", "_", regex=True)

# Step 3: Drop duplicates based on Subject ID + Research Group
mri_df = mri_df.drop_duplicates(subset=['Subject ID', 'Research Group'])

# Step 4: Map label names to integers
label_map = {'CN': 0, 'SMC': 1, 'EMCI': 2, 'MCI': 3, 'LMCI': 4, 'AD': 5}
mri_df['Labels'] = mri_df['Research Group'].map(label_map)

# Step 5: Merge on IID / Subject ID
merged_df = pd.merge(mri_df[['Subject ID', 'Labels']], snp_df, how='inner', left_on='Subject ID', right_on='IID')

# Step 6: Drop redundant IID column
merged_df = merged_df.drop(columns=['IID'])

# Step 7: Drop columns with any NaNs (only feature columns)
feature_cols = [col for col in merged_df.columns if col not in ['Subject ID', 'Labels']]
clean_features = merged_df[feature_cols].dropna(axis=1, how='any')
merged_df = pd.concat([merged_df[['Subject ID']], clean_features, merged_df[['Labels']]], axis=1)

# Step 8: Save final CSV
merged_df.to_csv(output_file, index=False)
print(f"Saved merged data to: {output_file}")

Saved merged data to: /home/piyali/Piyali/Data/SNP/Subject_feature_label_SNP_final.csv


In [77]:
merged_df.info

<bound method DataFrame.info of       Subject ID  rs10155688_A  rs10502267_A  ...  rs4542114_G  rs4898398_A  Labels
0     002_S_0295             1             0  ...            0            0       0
1     002_S_0413             1             0  ...            0            0       0
2     002_S_0559             1             0  ...            0            0       0
3     002_S_0685             0             0  ...            2            1       0
4     002_S_0729             0             0  ...            1            0       3
...          ...           ...           ...  ...          ...          ...     ...
1051  941_S_4376             1             0  ...            0            0       0
1052  941_S_4377             0             0  ...            1            1       4
1053  941_S_4764             0             0  ...            2            0       2
1054  941_S_5124             0             0  ...            1            0       1
1055  941_S_5193             0             0

In [78]:
merged_df.head()

,Subject ID,rs10155688_A,rs10502267_A,rs1079841_A,rs11611755_T,rs12209455_G,rs12422489_A,rs130743_A,rs1446931_T,rs17257_A,rs17282_A,rs17499655_T,rs17767132_G,rs1997739_C,rs2004892_G,rs2026726_A,rs2088685_T,rs2213198_T,rs2241015_T,rs2291715_T,rs2338696_T,rs2477648_A,rs2855983_A,rs2858016_T,rs2871771_C,rs2895461_A,rs2918515_T,rs361473_T,rs3855349_C,rs4364877_A,rs4821082_T,rs4968723_G,rs562790_A,rs6421025_T,rs6913464_T,rs6916246_C,rs6937363_T,rs714215_T,rs722343_G,rs7290901_A,...,rs4330820_A,rs5924683_C,rs6627462_T,rs1882713_C,rs7054854_C,rs1467884_A,rs210567_C,rs11094547_T,rs389292_T,rs2301188_C,rs916043_A,rs12690074_A,rs2071261_A,rs1894356_T,rs2071262_T,rs5925265_A,rs5970431_G,rs5945306_C,rs5945320_A,rs6571298_G,rs2266862_T,rs1126499_T,rs743642_T,rs2285034_C,rs12010175_A,rs4477197_T,rs869131_A,rs5945326_G,rs5945157_T,rs5987128_T,rs7877342_C,rs2734647_T,rs2071649_G,rs2239466_G,rs2070822_G,rs2283762_G,rs11156577_C,rs4542114_G,rs4898398_A,Labels
0,002_S_0295,1,0,2,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,2,1,1,2,1,0,0,0,1,1,0,0,0,0,0,0,0,1,...,0,2,2,0,2,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,002_S_0413,1,0,1,0,0,0,1,2,0,1,0,1,0,1,0,2,0,0,1,1,0,0,1,1,0,0,0,2,1,0,0,1,1,0,0,0,1,0,2,...,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,1,1,1,1,1,0,1,1,2,0,0,1,1,0,0,0,0,0,0,0
2,002_S_0559,1,0,0,1,0,0,2,1,0,0,0,0,0,0,0,0,1,1,0,1,2,2,1,0,1,1,0,0,2,2,0,2,1,0,1,0,0,0,2,...,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,0,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,002_S_0685,0,0,0,0,0,0,1,1,0,0,0,1,0,1,1,0,0,0,0,1,0,1,0,0,0,1,0,1,1,2,2,0,2,0,0,0,0,1,0,...,0,2,2,0,0,0,0,0,0,0,1,0,0,2,0,0,0,0,0,2,1,0,0,0,0,0,1,1,1,0,2,2,2,2,2,0,0,2,1,0
4,002_S_0729,0,0,1,0,1,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,2,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,1,1,...,0,2,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,3


In [79]:
# To check if any subject has more than one label or not:
len(merged_df['Subject ID']), len(set(merged_df['Subject ID']))

(1056, 1056)

In [80]:
# If the alleles need to be removed:
output_file_no_alleles = "/home/piyali/Piyali/Data/SNP/subject_features_labels_without_allels.csv"

merged_df.rename(columns=lambda x: x.replace('_A', '').replace('_C', '').replace('_G', '').replace('_T', ''), inplace=True)
print(merged_df.columns)

Index(['Subject ID', 'rs10155688', 'rs10502267', 'rs1079841', 'rs11611755',
       'rs12209455', 'rs12422489', 'rs130743', 'rs1446931', 'rs17257',
       ...
       'rs7877342', 'rs2734647', 'rs2071649', 'rs2239466', 'rs2070822',
       'rs2283762', 'rs11156577', 'rs4542114', 'rs4898398', 'Labels'],
      dtype='object', length=73703)


In [81]:
len(merged_df.columns), len(set(merged_df.columns))

(73703, 73703)

In [82]:
cn_mask = merged_df['Labels'] == 5
cn_indices = merged_df.index[cn_mask].tolist()
len(cn_indices)

182

### Labels count:
CN = 274 \
SMC = 54 \
EMCI = 200 \
MCI = 245 \
LMCI = 101 \
AD = 182 

### Take 4 classes as: 
CN : 274, \
EMCI (= SMC + EMCI) : 254, \
MCI : 245, \
AD (= LMCI + AD) : 282

In [83]:
# Remap 6-class labels (0 to 5) into 4-class labels (0 to 3)
label_4class_map = {
    0: 0,  # CN
    1: 1,  # SMC → EMCI
    2: 1,  # EMCI → EMCI
    3: 2,  # MCI
    4: 3,  # LMCI → AD
    5: 3   # AD → AD
}

# Create a new variable with 4-class labels
labels_4class = merged_df['Labels'].map(label_4class_map)

# Example: get feature matrix (excluding Subject ID and original Labels)
features = merged_df.drop(columns=['Subject ID', 'Labels'])